In [4]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *

from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

conf=SparkConf()
# conf.set("spark.executor.memory", "2g")
# conf.set("spark.driver.memory", "12g")
# conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

In [5]:
import sys
sys.path.append('/home/preacher/Bungee/CodeRepo/match-management-system/')

In [6]:

from aw_to_mw_synchronizer.runner import Synchronizer

In [7]:
args = {
    "customers" : 
        {
            "grocery": ["imperfect<>imperfect", "wholefoods<>wholefoods"],
            "office_supplies" : ["staples<>staples"],
            "pets": ["petco<>petco", "chewy<>chewy"],
            "sports_outdoors": [],
            "petsca" : []
        },
    "last_stack_run_date" : "2023-12-30"
        
}

In [8]:
# Synchronizer(spark, args, "dev", None).sync()

In [9]:


from aw_to_mw_synchronizer.transformer.bungee_audit_match_processor import BungeeAuditMatchProcessor
from aw_to_mw_synchronizer.transformer.customer_audit_match_processor import CustomerAuditMatchProcessor
from aw_to_mw_synchronizer.transformer.match_merger import MatchMerger


In [10]:
# fastlane_successful = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/fastlane_successful.csv")
# fastlane_successful.show()

# fastlane_unsuccessful = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/fastlane_unsuccessful.csv")
# fastlane_unsuccessful.show()

# match_library = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/customer_audit_library.csv")
# match_library.show()

In [11]:
# bungee_audited_matches = BungeeAuditMatchProcessor(args, fastlane_successful, fastlane_unsuccessful).process()

In [12]:
# customer_audited_matches = CustomerAuditMatchProcessor(args, match_library).process()

In [13]:
def _change_schema(mw):
        mw = mw.\
                withColumn("pair_id", col("pair_id").cast("string") ).\
                withColumn("segment", col("segment").cast("string") ).\
                withColumn("base_sku_uuid", col("base_sku_uuid").cast("string") ).\
                withColumn("comp_sku_uuid", col("comp_sku_uuid").cast("string") ).\
                withColumn("base_source_store", col("base_source_store").cast("string") ).\
                withColumn("comp_source_store", col("comp_source_store").cast("string") ).\
                withColumn("bungee_audit_status", col("bungee_audit_status").cast("string") ).\
                withColumn("misc_info", col("misc_info").cast("string") ).\
                withColumn("bungee_auditor", col("bungee_auditor").cast("string") ).\
                withColumn("bungee_audit_date", col("bungee_audit_date").cast("date") ).\
                withColumn("bungee_auditor_comment", col("bungee_auditor_comment").cast("string") ).\
                withColumn("seller_type", col("seller_type").cast("string") ).\
                withColumn("client_audit_status_l1", col("client_audit_status_l1").cast("string") ).\
                withColumn("client_auditor_l1", col("client_auditor_l1").cast("string") ).\
                withColumn("client_audit_date_l1", col("client_audit_date_l1").cast("date") ).\
                withColumn("client_auditor_l1_comment", col("client_auditor_l1_comment").cast("string") ).\
                withColumn("client_audit_status_l2", col("client_audit_status_l2").cast("string") ).\
                withColumn("client_auditor_l2", col("client_auditor_l2").cast("string") ).\
                withColumn("client_audit_date_l2", col("client_audit_date_l2").cast("date") ).\
                withColumn("client_auditor_l2_comment", col("client_auditor_l2_comment").cast("string") ).\
                withColumn("created_date", col("created_date").cast("date") ).\
                withColumn("created_by", col("created_by").cast("string") ).\
                withColumn("updated_date", col("updated_date").cast("date") ).\
                withColumn("workflow_name", col("workflow_name").cast("date") ).\
                withColumn("updated_by", col("updated_by").cast("string") )
        return mw

In [14]:
# customer_audited_matches = _change_schema(customer_audited_matches)
# bungee_audited_matches = _change_schema(bungee_audited_matches)

In [15]:
# customer_audited_matches.printSchema()

fastlane and match library CONFLICT
1. only difference in client status and bungee audit status of ml and same bungee audit status for fastlane
2. diffetent bungee state for ml and fastlane
3. diffetent client and bungee status
3. all same

In [16]:
# customer_audited_matches = customer_audited_matches.select("*", explode("match_source_score_map").alias("source", "score")).drop("match_source_score_map")
# customer_audited_matches.coalesce(1).write.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/customer_audited_matches")
# bungee_audited_matches = bungee_audited_matches.select("*", explode("match_source_score_map").alias("source", "score")).drop("match_source_score_map")
# bungee_audited_matches.coalesce(1).write.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/bungee_audited_matches")

In [17]:
# audit_match = CustomerAuditMatchProcessor(args, audit_library).process()

In [18]:
# audit_match.show()

In [19]:
bungee_audited = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/bungee_audited_matches")
bungee_audited = bungee_audited.withColumn("match_source_score_map", create_map(col("source"), col("score"))).drop("source", "score")
bungee_audited.show()

customer_audited = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/customer_audited_matches")
customer_audited = customer_audited.withColumn("match_source_score_map", create_map(col("source"), col("score"))).drop("source", "score")
customer_audited.show()

mw = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/mw")
mw = mw.withColumn("match_source_score_map", create_map(col("source"), col("score"))).drop("source", "score")
mw.show()

23/12/28 13:58:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------+--------------------+--------------------+--------------------+----------------------+-----------------+-------------------+-----------------+-------------------------+-----------------+-------------------------+--------------------+--------------------+----------------------+----------------------+--------------------+--------------------+----------+------------+---------+--------------------+---------------+-----------+----------+------------+-------------+--------+----------------------+
|aggregated_score|       base_sku_uuid|   base_source_store|      bungee_auditor|bungee_auditor_comment|bungee_audit_date|bungee_audit_status|client_auditor_l1|client_auditor_l1_comment|client_auditor_l2|client_auditor_l2_comment|client_audit_date_l1|client_audit_date_l2|client_audit_status_l1|client_audit_status_l2|       comp_sku_uuid|   comp_source_store|created_by|created_date|misc_info|             pair_id|        segment|seller_type|updated_by|updated_date|workflow_name|prior

In [20]:
bungee_audited_match = MatchMerger("dev", args, bungee_audited, customer_audited, mw).merge_matches()

Combining dfs
converting undirected to directed pairs
Combining dfs
undirected matches count  83
+-----------------+-----------------------------------------+------------------------------+------------------------------------+----------------------+-----------------+-------------------+-----------------------------+-------------------------+---------------------+---------------------------------------------------+--------------------+--------------------+----------------------+----------------------+-----------------------------------------------------+----------------------------------------+----------+------------+---------+-------------------------------------------------------------------------+---------------+-----------+----------+------------+-------------+--------+-------------------------------+--------------------------+
|aggregated_score |base_sku_uuid                            |base_source_store             |bungee_auditor                      |bungee_auditor_comment|bunge

In [21]:
bungee_audited_match.filter(col("bungee_audit_status") == "CONFLICT").select("bungee_audit_status", "client_audit_status_l1", "client_audit_status_l2").show(n = 200)

+-------------------+----------------------+----------------------+
|bungee_audit_status|client_audit_status_l1|client_audit_status_l2|
+-------------------+----------------------+----------------------+
|           CONFLICT|           NOT_A_MATCH|                  null|
|           CONFLICT|           EXACT_MATCH|                  null|
|           CONFLICT|           EXACT_MATCH|                  null|
|           CONFLICT|         SIMILAR_MATCH|         SIMILAR_MATCH|
|           CONFLICT|         SIMILAR_MATCH|           NOT_A_MATCH|
|           CONFLICT|           EXACT_MATCH|           NOT_A_MATCH|
|           CONFLICT|                  null|                  null|
|           CONFLICT|                  null|                  null|
+-------------------+----------------------+----------------------+



In [22]:
bungee_status = bungee_audited.select("pair_id", col("bungee_audit_status").alias("bungee_audit_status_in_fastlane")).withColumn("fastlane", lit("y"))
customer_status = customer_audited.select("pair_id", col("bungee_audit_status").alias("bungee_audit_status_in_ml")).withColumn("ml", lit("y"))
old_status = bungee_status.join(customer_status, "pair_id", "inner")
old_status.show()

+--------------------+-------------------------------+--------+-------------------------+---+
|             pair_id|bungee_audit_status_in_fastlane|fastlane|bungee_audit_status_in_ml| ml|
+--------------------+-------------------------------+--------+-------------------------+---+
|261930<>chewy<>ch...|                    NOT_A_MATCH|       y|              EXACT_MATCH|  y|
|148673<>chewy<>ch...|                    EXACT_MATCH|       y|              EXACT_MATCH|  y|
|70151<>chewy<>che...|                    EXACT_MATCH|       y|              EXACT_MATCH|  y|
+--------------------+-------------------------------+--------+-------------------------+---+



In [23]:
new_status = bungee_audited_match.select("pair_id","bungee_audit_status", "client_audit_status_l1", "client_audit_status_l2")
abc = new_status.join(bungee_status, "pair_id", "left").filter(col("bungee_audit_status") == "CONFLICT")
abc.show(truncate=False)
abc.join(customer_status, "pair_id", "left").filter(col("bungee_audit_status") == "CONFLICT").show(truncate=False)

+--------------------------------------------------------------+-------------------+----------------------+----------------------+-------------------------------+--------+
|pair_id                                                       |bungee_audit_status|client_audit_status_l1|client_audit_status_l2|bungee_audit_status_in_fastlane|fastlane|
+--------------------------------------------------------------+-------------------+----------------------+----------------------+-------------------------------+--------+
|1055542<>petco<>petco_47313<>petcarerx<>petcarerx             |CONFLICT           |NOT_A_MATCH           |null                  |null                           |null    |
|1268942<>bsnsports<>bsnsports_10-394<>gophersport<>gophersport|CONFLICT           |EXACT_MATCH           |null                  |null                           |null    |
|1272826<>bsnsports<>bsnsports_42-561<>gophersport<>gophersport|CONFLICT           |EXACT_MATCH           |null                  |null      